In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np

import acquire_mall

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree

import scipy.stats as stats
from scipy.stats import pearsonr

import graphviz
from graphviz import Graph

import env

In [2]:
from acquire_mall import get_connection
from acquire_mall import get_mall_customers_data

In [3]:
df = get_mall_customers_data()
df.shape

(200, 5)

In [4]:
pd.concat([df.head(), df.tail()])

,customer_id,gender,age,annual_income,spending_score
0,1,Male,19,15,39
1,2,Male,21,15,81
2,3,Female,20,16,6
3,4,Female,23,16,77
4,5,Female,31,17,40
195,196,Female,35,120,79
196,197,Female,45,126,28
197,198,Male,32,126,74
198,199,Male,32,137,18
199,200,Male,30,137,83


In [5]:
df.describe()

,customer_id,age,annual_income,spending_score
count,200.000000,200.000000,200.000000,200.000000
mean,100.500000,38.850000,60.560000,50.200000
std,57.879185,13.969007,26.264721,25.823522
min,1.000000,18.000000,15.000000,1.000000
25%,50.750000,28.750000,41.500000,34.750000
50%,100.500000,36.000000,61.500000,50.000000
75%,150.250000,49.000000,78.000000,73.000000
max,200.000000,70.000000,137.000000,99.000000


In [6]:
def encode_gender(df):
    encoder = LabelEncoder()
    encoder.fit(df.gender)
    df = df.assign(gender_encoded = encoder.transform(df.gender))
    return df

In [7]:
df = encode_gender(df)

In [8]:
def encode_object(df):
    for col in df.select_dtypes('object'):
        encoder = LabelEncoder()
        encoder.fit(df[col])
        df[col + '_encoded'] = df.assign(encoder.transform(df[col]))
    return df

In [9]:
df = encode_object(df)

TypeError: assign() takes 1 positional argument but 2 were given

In [150]:
df.head()

,customer_id,gender,age,annual_income,spending_score,gender_encoded
0,1,Male,19,15,39,1
1,2,Male,21,15,81,1
2,3,Female,20,16,6,0
3,4,Female,23,16,77,0
4,5,Female,31,17,40,0


In [10]:
def get_outliers(df, k):
    q1, q3 = df.quantile([.25, .75])
    iqr = q3 - q1
    upper_bound = q3 + 1.5 * iqr
    lower_bound = q1 - 1.5 * iqr
    upper = df.apply(lambda x: max([x - upper_bound, 0]))
    lower = df.apply(lambda x: min([x + lower_bound, 0]))
    return(upper, lower)

In [11]:
def add_outlier_columns(df, k):
    for col in df.select_dtypes('number'):
        df[col + '_upper_outliers'] = get_upper_outliers(df[col], k)
        df[col + '_lower_outliers'] = get_lower_outliers(df[col], k)
    return df

In [12]:
df = add_outlier_columns(df, 1.5)
pd.concat([df.head(), df.tail()])

NameError: name 'get_upper_outliers' is not defined